<a href="https://colab.research.google.com/github/Seong-jieun/Personal_Project/blob/main/StudyCafe_Usage_analysis_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

- 분석 배경
    - 취업 준비를 위해 스터디카페를 이용 중인데 비용이 꽤 많이 드는 것 같아서 적절한 이용권과 이용시간을 계산해 볼 필요가 있다는 생각이 들었다.

- 분석 목적
    - 이용권을 합리적으로 이용하려면 하루 최소 몇 시간을 이용하면 되는지 계산한다.
    - 결제한 이용권이 나의 스터디카페 이용패턴에 맞는 이용권인지 확인한다.

- 데이터 정보
    - 데이터 수집 기간: 2024-12-24 ~ 2025-03-10
    - 수집 데이터: 구매 이용권, 이용일자, 입실시간, 퇴실시간, 결제금액, 이용권별 가격
    - 수집 방법:
        - 스터디카페 앱 내 입실, 퇴실 알림, 결제내역을 구글 스프레드시트에 수기 입력했다.
        - 외출 여부, 시간은 앱 내에서 확인이 안 되기 때문에 이용시간 내 교통카드 이용내역, 외부에서 결제한 카드내역을 바탕으로 체크했고, 카드 이용 내역이 없는 외출의 경우(ex. 산책) 다이어리 기록, 기억에 의존하여 외출 여부만 체크했다.


# 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import math
import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [2]:
usedata = pd.read_csv('/content/drive/MyDrive/personal_da_project/스터디카페 이용기록 - 이용기록.csv')    # 스터디카페 이용기록
payment = pd.read_csv('/content/drive/MyDrive/personal_da_project/스터디카페 이용기록 - 결제정보.csv')    # 스터디카페 결제정보
price_table = pd.read_csv('/content/drive/MyDrive/personal_da_project/스터디카페 이용기록 - 가격표.csv')  # 이용권 가격정보

In [3]:
usedata.head()

,이용권,날짜,입실시간,퇴실시간,외출여부
0,시간권_4시간_1,2024-12-24,13:21:28,17:21:28,x
1,시간권_4시간_2,2024-12-27,13:59:13,17:59:13,x
2,시간권_50시간_1,2024-12-30,13:45:57,19:33:57,x
3,시간권_50시간_1,2025-01-02,11:22:07,19:52:33,x
4,시간권_50시간_1,2025-01-03,11:03:36,18:09:16,x


In [4]:
payment

,결제이용권,결제일자,시작일시,종료일시,결제금액,이용권금액,비고
0,시간권_4시간_1,2024-12-24,2024-12-24 13:21:00,2024-12-24 17:21:00,"5,000","5,000",NaN
1,시간권_4시간_2,2024-12-27,2024-12-27 13:59:00,2024-12-27 17:59:00,"5,000","5,000",NaN
2,시간권_50시간_1,2024-12-30,2024-12-30 13:45:00,2025-01-13 11:22:00,"70,000","70,000",NaN
3,기간권_4주_1,2025-01-13,2025-01-13 11:21:00,2025-02-10 11:21:00,"130,000","130,000",NaN
4,기간권_4주_2,2025-02-09,2025-02-10 11:21:00,2025-03-10 11:21:00,"120,000","120,000",연장결제
5,기간권_4주_3,2025-03-05,2025-03-10 11:21:00,2025-03-10 11:21:00,"99,000","120,000","연장결제, 할인이벤트, 쿠폰적용"


In [5]:
price_table

,이용권 구분,이용권,가격,동일권종으로 연장시 가격
0,기간권,2주(14일),"80,000","75,000"
1,기간권,4주(28일),"130,000","120,000"
2,기간권,12주(84일),"350,000","340,000"
3,시간권,50시간(8주),"70,000","65,000"
4,시간권,100시간(16주),"120,000","110,000"
5,시간권,200시간(32주),"220,000","210,000"
6,당일권,2시간,"3,000","3,000"
7,당일권,4시간,"5,000","5,000"
8,당일권,6시간,"6,000","6,000"
9,당일권,8시간,"8,000","8,000"


[컬럼 정보]
- usedata
    - 이용권: 이용권 종류
    - 날짜: 이용 일자
    - 입실시간: 입실한 시간
    - 퇴실시간: 퇴실한 시간
    - 외출여부: 외출한 경우 o, 외출하지 않은 경우 x (기간권 사용 시에만 외출함)

- payment
    - 결제이용권: 결제한 이용권 종류
    - 결제일자: 이용권을 결제한 일자
    - 시작일시: 이용권 적용 시작 일시
    - 종료일시: 이용권 적용 종료 일시
    - 결제금액: 실제 결제한 금액
    - 이용권금액: 이용권 판매 금액
    - 비고: 참고사항

- price_table
    - 이용권 구분: 이용권은 기간권, 시간권, 당일권으로 구분됨
    - 이용권: 이용권 이름 (시간권은 괄호 안의 기간 안에 시간을 소진해야 함)
    - 가격: 이용권 가격
    - 동일권종으로 연장시 가격: 이용권 연장 구매할 경우 가격 (동일권종은 기간권->기간권, 시간권->시간권일 경우를 의미함)

# 데이터 전처리

## 데이터 타입 확인

In [6]:
usedata.info(), usedata.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이용권     57 non-null     object
 1   날짜      57 non-null     object
 2   입실시간    56 non-null     object
 3   퇴실시간    57 non-null     object
 4   외출여부    57 non-null     object
dtypes: object(5)
memory usage: 2.4+ KB


(None, (57, 5))

In [7]:
payment.info(), payment.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   결제이용권   6 non-null      object
 1   결제일자    6 non-null      object
 2   시작일시    6 non-null      object
 3   종료일시    6 non-null      object
 4   결제금액    6 non-null      object
 5   이용권금액   6 non-null      object
 6   비고      2 non-null      object
dtypes: object(7)
memory usage: 468.0+ bytes


(None, (6, 7))

In [8]:
price_table.info(), price_table.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   이용권 구분         11 non-null     object
 1   이용권            11 non-null     object
 2   가격             11 non-null     object
 3   동일권종으로 연장시 가격  11 non-null     object
dtypes: object(4)
memory usage: 484.0+ bytes


(None, (11, 4))

## 데이터 타입 변경

In [9]:
# usedata 테이블 데이터 타입 변경
usedata['날짜'] = pd.to_datetime(usedata['날짜'], format='%Y-%m-%d')
usedata['입실시간'] = pd.to_datetime(usedata['입실시간'], format='%H:%M:%S', errors='coerce')
usedata['퇴실시간'] = pd.to_datetime(usedata['퇴실시간'], format='%H:%M:%S', errors='coerce')

In [10]:
# 변경된 타입 확인
usedata.dtypes

,0
이용권,object
날짜,datetime64[ns]
입실시간,datetime64[ns]
퇴실시간,datetime64[ns]
외출여부,object


In [11]:
# 변경된 테이블 확인
usedata.head()

# 입실시간, 퇴실시간에 '1900-01-01'가 같이 출력되지만, 분석에는 시간 데이터만 필요하므로 따로 전처리하지 않기로 한다

,이용권,날짜,입실시간,퇴실시간,외출여부
0,시간권_4시간_1,2024-12-24,1900-01-01 13:21:28,1900-01-01 17:21:28,x
1,시간권_4시간_2,2024-12-27,1900-01-01 13:59:13,1900-01-01 17:59:13,x
2,시간권_50시간_1,2024-12-30,1900-01-01 13:45:57,1900-01-01 19:33:57,x
3,시간권_50시간_1,2025-01-02,1900-01-01 11:22:07,1900-01-01 19:52:33,x
4,시간권_50시간_1,2025-01-03,1900-01-01 11:03:36,1900-01-01 18:09:16,x


In [12]:
# payment 테이블 데이터 타입 변경
payment['결제금액'] = payment['결제금액'].str.replace(',', '')
payment['결제금액'] = payment['결제금액'].astype(int)
payment['이용권금액'] = payment['이용권금액'].str.replace(',', '')
payment['이용권금액'] = payment['이용권금액'].astype(int)

In [13]:
# 변경된 타입 확인
payment.dtypes

,0
결제이용권,object
결제일자,object
시작일시,object
종료일시,object
결제금액,int64
이용권금액,int64
비고,object


In [14]:
# price_table 테이블 데이터 타입 변경
price_table['가격'] = price_table['가격'].str.replace(',', '')
price_table['가격'] = price_table['가격'].astype(int)
price_table['동일권종으로 연장시 가격'] = price_table['동일권종으로 연장시 가격'].str.replace(',', '')
price_table['동일권종으로 연장시 가격'] = price_table['동일권종으로 연장시 가격'].astype(int)

# 컬럼명 변경
price_table.rename(columns={'동일권종으로 연장시 가격':'연장가'}, inplace=True)

In [15]:
# 변경된 타입 확인
price_table.dtypes

,0
이용권 구분,object
이용권,object
가격,int64
연장가,int64


## 결측치 처리

In [16]:
# 결측값이 있는 행 확인
usedata[usedata.isnull().any(axis=1)]

,이용권,날짜,입실시간,퇴실시간,외출여부
31,기간권_4주_1,2025-02-06,NaT,1900-01-01 19:09:47,o


- 결측값 정보
    - 데이터 수집 과정에서 실수로 삭제하면서 입력하지 못한 '입실시간' 값이다.

- 결측값 처리 방법
    - `기간권_4주_1`에 해당하면서, `15시 이전에 입실`한 경우의 `평균 입실시간`으로 결측값을 대체하기로 한다.

In [17]:
# '기간권_4주_1'에 해당하는 데이터 필터링
grouped = usedata[usedata['이용권'] == '기간권_4주_1']

# 15시 이전의 입실시간만 필터링 (시간이 15:00:00 이전인 데이터만 선택)
grouped_before_3pm = grouped[grouped['입실시간'].dt.strftime('%H:%M:%S') < '15:00:00']

# 15시 이전 입실시간의 평균 구하기 (시분초까지 포함)
mean_time = grouped_before_3pm['입실시간'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second).mean()

# 평균을 시분초로 다시 변환
mean_hour = mean_time // 3600
mean_minute = (mean_time % 3600) // 60
mean_second = mean_time % 60

# 평균 시간으로 null 값 대체 (시분초 형식으로)
usedata['입실시간'] = usedata.apply(
    lambda row: pd.to_datetime(f"1900-01-01 {int(mean_hour)}:{int(mean_minute)}:{int(mean_second)}")
    if pd.isnull(row['입실시간']) and row['이용권'] == '기간권_4주_1' else row['입실시간'],
    axis=1
)

In [18]:
# 2025-02-06 일자 행 확인
usedata[usedata['날짜'] == '2025-02-06']

# 입실시간: 11:41:08

,이용권,날짜,입실시간,퇴실시간,외출여부
31,기간권_4주_1,2025-02-06,1900-01-01 11:41:08,1900-01-01 19:09:47,o


## 파생변수 생성 - 이용시간

In [19]:
# 이용시간 계산
usedata['이용시간'] = usedata['퇴실시간'] - usedata['입실시간']
usedata.head()

,이용권,날짜,입실시간,퇴실시간,외출여부,이용시간
0,시간권_4시간_1,2024-12-24,1900-01-01 13:21:28,1900-01-01 17:21:28,x,0 days 04:00:00
1,시간권_4시간_2,2024-12-27,1900-01-01 13:59:13,1900-01-01 17:59:13,x,0 days 04:00:00
2,시간권_50시간_1,2024-12-30,1900-01-01 13:45:57,1900-01-01 19:33:57,x,0 days 05:48:00
3,시간권_50시간_1,2025-01-02,1900-01-01 11:22:07,1900-01-01 19:52:33,x,0 days 08:30:26
4,시간권_50시간_1,2025-01-03,1900-01-01 11:03:36,1900-01-01 18:09:16,x,0 days 07:05:40


# EDA

In [67]:
# 데이터 세트 전체 기간
print(f'이 데이터 세트는 {min(usedata["날짜"].dt.date)}부터 {max(usedata["날짜"].dt.date)}까지의 데이터입니다.')

# 구매한 적 있는 이용권
print(f'구매한 적 있는 이용권은 {payment["결제이용권"].unique()}로 총 {payment["결제이용권"].nunique()}종류입니다.')

# 이용권 구매 횟수와 금액
print(f'이용권 구매 횟수는 {payment["결제이용권"].value_counts().sum()}회 입니다.')
print(f'이용권 총 구매 금액은 {payment["결제금액"].sum()}입니다.')

이 데이터 세트는 2024-12-24부터 2025-03-10까지의 데이터입니다.
구매한 적 있는 이용권은 ['시간권_4시간_1' '시간권_4시간_2' '시간권_50시간_1' '기간권_4주_1' '기간권_4주_2' '기간권_4주_3']로 총 6종류입니다.
이용권 구매 횟수는 6회 입니다.
이용권 총 구매 금액은 429000입니다.


In [69]:
# 이용권별 이용기간, 일 수 확인
def ticket_usage_info(df, ticket):
    min_date = min(df[df["이용권"] == ticket]["날짜"].dt.date)
    max_date = max(df[df["이용권"] == ticket]["날짜"].dt.date)
    days = df[df["이용권"] == ticket].value_counts().sum()
    return print(f'{ticket}의 이용기간은 {min_date}부터 {max_date}까지이며 {days}일 이용했습니다.')

def days_used(df, ticket):
    days = df[df["이용권"] == ticket].value_counts().sum()
    return days

In [70]:
ticket_usage_info(usedata, '시간권_50시간_1')
ticket_usage_info(usedata, '기간권_4주_1')
ticket_usage_info(usedata, '기간권_4주_2')

시간권_50시간_1의 이용기간은 2024-12-30부터 2025-01-13까지이며 9일 이용했습니다.
기간권_4주_1의 이용기간은 2025-01-13부터 2025-02-09까지이며 23일 이용했습니다.
기간권_4주_2의 이용기간은 2025-02-10부터 2025-03-10까지이며 22일 이용했습니다.


In [71]:
# 이용권 총 이용시간
def total_usetime(df, ticket):
    total_time = df[df["이용권"] == ticket]["이용시간"].sum()

    # 시:분:초 형식으로 변환
    total_seconds = int(total_time.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60

    # 시:분:초 형식으로 출력
    total_format = f"{hours:02d}:{minutes:02d}:{seconds:02d}"

    return total_format

In [72]:
# 이용권 평균 이용시간
def avg_usetime(df, ticket):
    avg_time = str(df[df["이용권"] == ticket]["이용시간"].mean()).split(" ")[-1].split(".")[0]
    return avg_time

# 이용권 평균 입실시각
def avg_intime(df, ticket):
    avg_in = str(df[df["이용권"] == ticket]["입실시간"].mean()).split(" ")[-1].split(".")[0]
    return avg_in

# 이용권 평균 퇴실시각
def avg_outtime(df, ticket):
    avg_out = str(df[df["이용권"] == ticket]["퇴실시간"].mean()).split(" ")[-1].split(".")[0]
    return avg_out

In [75]:
print('시간권_50시간_1의 총 이용시간:', total_usetime(usedata, "시간권_50시간_1"), '평균 이용시간:', avg_usetime(usedata, "시간권_50시간_1"), '평균 입실시각:', avg_intime(usedata, "시간권_50시간_1"), '평균 퇴실시각:', avg_outtime(usedata, "시간권_50시간_1"))
print('기간권_4주_1의 총 이용시간:', total_usetime(usedata, "기간권_4주_1"), '평균 이용시간:', avg_usetime(usedata, "기간권_4주_1"), '평균 입실시각:', avg_intime(usedata, "기간권_4주_1"), '평균 퇴실시각:', avg_outtime(usedata, "기간권_4주_1"))
print('기간권_4주_2의 총 이용시간:', total_usetime(usedata, "기간권_4주_2"), '평균 이용시간:', avg_usetime(usedata, "기간권_4주_2"), '평균 입실시각:', avg_intime(usedata, "기간권_4주_2"), '평균 퇴실시각:', avg_outtime(usedata, "기간권_4주_2"))

시간권_50시간_1의 총 이용시간: 50:04:26 평균 이용시간: 05:33:49 평균 입실시각: 12:15:18 평균 퇴실시각: 17:49:08
기간권_4주_1의 총 이용시간: 150:24:29 평균 이용시간: 06:32:22 평균 입실시각: 12:55:16 평균 퇴실시각: 19:27:38
기간권_4주_2의 총 이용시간: 137:56:49 평균 이용시간: 06:16:13 평균 입실시각: 11:36:17 평균 퇴실시각: 17:52:30


- 기간권_4주 출석률 확인

In [76]:
# 기간권_4주 출석률
def attendance_rate(df, ticket_name):
    days = 28
    attendance = df[df["이용권"] == ticket_name].value_counts().sum()
    return round(attendance / days * 100, 2)

In [78]:
print('기간권_4주_1의 출석률:', attendance_rate(usedata, '기간권_4주_1'))
print('기간권_4주_2의 출석률:', attendance_rate(usedata, '기간권_4주_2'))

기간권_4주_1의 출석률: 82.14
기간권_4주_2의 출석률: 78.57


- 외출 횟수

In [79]:
# 기간권 외출 횟수
def rest_count(df, ticket_name):
    rest = df[(df['이용권'] == ticket_name) & (df['외출여부'] == 'o')].value_counts().sum()
    return rest

In [80]:
print('기간권_4주_1의 외출 횟수:', rest_count(usedata, '기간권_4주_1'))
print('기간권_4주_2의 외출 횟수:', rest_count(usedata, '기간권_4주_2'))

기간권_4주_1의 외출 횟수: 8
기간권_4주_2의 외출 횟수: 4


- 전체기간의 평균 입퇴실시간

In [27]:
# 전체기간의 평균 입퇴실시간
in_avg = str(usedata['입실시간'].mean()).split(" ")[-1].split(".")[0]
out_avg = str(usedata['퇴실시간'].mean()).split(" ")[-1].split(".")[0]

print("전체기간 평균 입실시간:", in_avg)
print("전체기간 평균 퇴실시간:", out_avg)

전체기간 평균 입실시간: 12:18:24
전체기간 평균 퇴실시간: 18:31:04


- 요약된 데이터프레임 만들기

In [150]:
# HH:MM:SS 형식 문자열을 float로 변환 (검증 단계에서 사용하기 위함)
def convert_hhmmss_to_hours(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h + m/60 + s/3600

def create_summary_dataframe(df1, df2, tickets):    # df1: 이용기록 데이터, df2: 결제정보 데이터
    data = []

    for ticket in tickets:
        # 결제금액 추출
        payment_amount = df2[df2['결제이용권'] == ticket]['결제금액'].values[0]

        # 2) 원본 통계
        total_time_str = total_usetime(df1, ticket)      # HH:MM:SS
        avg_time_str   = avg_usetime(df1, ticket)        # HH:MM:SS
        intime_str     = avg_intime(df1, ticket)
        outtime_str    = avg_outtime(df1, ticket)
        used_days      = days_used(df1, ticket)
        attend_rate    = attendance_rate(df1, ticket)
        rest_cnt       = rest_count(df1, ticket)

        # 최소 총 이용시간, 일일 최소 이용시간 float 변환
        total_usetime_hours = convert_hhmmss_to_hours(total_time_str)
        avg_usetime_hours = convert_hhmmss_to_hours(avg_time_str)

        # 정보 집계
        data.append({
              '이용권': ticket
            , '총 이용시간': total_time_str
            , '평균 이용시간': avg_time_str
            , '평균 입실시간': intime_str
            , '평균 퇴실시간': outtime_str
            , '이용일수': used_days
            , '출석률(%, 기간권)': attend_rate
            , '외출횟수': rest_cnt
            , '결제금액': payment_amount
            , '총 이용시간_hours': total_usetime_hours
            , '평균 이용시간_hours': avg_usetime_hours
        })

    summary = pd.DataFrame(data)
    return summary

In [152]:
summary = create_summary_dataframe(usedata, payment, usedata['이용권'].unique())
summary

,이용권,총 이용시간,평균 이용시간,평균 입실시간,평균 퇴실시간,이용일수,"출석률(%, 기간권)",외출횟수,결제금액,총 이용시간_hours,평균 이용시간_hours
0,시간권_4시간_1,04:00:00,04:00:00,13:21:28,17:21:28,1,3.57,0,5000,4.000000,4.000000
1,시간권_4시간_2,04:00:00,04:00:00,13:59:13,17:59:13,1,3.57,0,5000,4.000000,4.000000
2,시간권_50시간_1,50:04:26,05:33:49,12:15:18,17:49:08,9,32.14,0,70000,50.073889,5.563611
3,기간권_4주_1,150:24:29,06:32:22,12:55:16,19:27:38,23,82.14,8,130000,150.408056,6.539444
4,기간권_4주_2,137:56:49,06:16:13,11:36:17,17:52:30,22,78.57,4,120000,137.946944,6.270278
5,기간권_4주_3,07:36:20,07:36:20,11:21:01,18:57:21,1,3.57,0,99000,7.605556,7.605556


[요약]
- 시간권을 사용할 때보다 기간권을 사용할 때 평균 이용시간이 약 1시간 더 늘었다.
- 기간권_4주_1과 기간권_4주_2는 이용일수, 총 이용시간, 평균 이용시간 패턴이 유사하다.
- 기간권 사용 첫 달(기간권_4주_1)보다 두번째 달(기간권_4주_2)에 총 이용시간과 출석률은 낮아졌다. (두번째 달에 하루 덜 갔기 때문으로 보인다.)

- 전체기간 평균 입실시간은 오전 12시 19분이고 평균 퇴실시간은 오후 18시 38분이다.

# 분석

- 분석 목적
    - 결제한 이용권이 나의 스터디카페 이용패턴에 합리적인 이용권인지 확인한다.
    - 이용권을 합리적으로 이용하려면 하루 최소 몇 시간을 이용하면 되는지 계산한다.


## 이용권 대비 이용시간 비교

- 9일동안 50시간을 이용하는 패턴이라면 예상되는 4주 동안 이용시간은 얼마일가?
- 계산식: `28일 X 50시간 / 9일`

In [32]:
# 28일 X 50시간 / 9일
28 * 50 / 9

155.55555555555554

- 계산결과 9일동안 50시간을 이용하는 패턴으로 4주 기간권을 쓰는 동안 155시간 30분을 이용할 것으로 예상할 수 있다.
- 실제 총 이용시간은 `기간권_4주_1`은 150시간 24분, `기간권_4주_2`는 137시간 56분으로 155시간 30분에 못 미친다.

- 금액적으로 비교해보기로 한다.
- 4주(28일)은 9일의 약 3.1배이므로 50시간권을 3번 쓸 경우와 4주 기간권을 한 번 쓸 경우 가격을 비교해본다.
    - 계산식: `50시간권 최초가격 + 연장시 가격 X 2`

In [33]:
# 50시간권 최초가격 + 연장시 가격 X 2
70000 + 65000 * 2
# 200,000원

200000

-> 50시간권을 3번 썼을 경우 예상되는 비용은 20만원이고 4주 기간권은 13만원이므로, 4주 기간권을 선택한 것은 합리적이었던 것으로 보인다.

## 이용권 가격대비 최소 이용시간 계산

- 앞으로 이용할 것으로 예상되는 이용권 위주로 이용금액 환산
    - 기간권 1일 이용금액
    - 시간권 1시간 이용금액


In [34]:
price_table

,이용권 구분,이용권,가격,연장가
0,기간권,2주(14일),80000,75000
1,기간권,4주(28일),130000,120000
2,기간권,12주(84일),350000,340000
3,시간권,50시간(8주),70000,65000
4,시간권,100시간(16주),120000,110000
5,시간권,200시간(32주),220000,210000
6,당일권,2시간,3000,3000
7,당일권,4시간,5000,5000
8,당일권,6시간,6000,6000
9,당일권,8시간,8000,8000


In [35]:
# 기간권 이용금액 환산 함수
w2 = 14
w4 = 28
w12 = 84

def p_convert_amount(df, ticket_name, days):
    price = int(math.ceil(price_table[price_table['이용권']==ticket_name]['가격'] / days))
    return price

def p_convert_amount_ex(df, ticket_name, days):
    price_extension = int(math.ceil(price_table[price_table['이용권']==ticket_name]['연장가'] / days))
    return price_extension

In [36]:
print(f'2주(14일): {p_convert_amount(price_table, "2주(14일)", w2)}원')
print(f'4주(28일): {p_convert_amount(price_table, "4주(28일)", w4)}원')
print(f'12주(84일): {p_convert_amount(price_table, "12주(84일)", w12)}원')

2주(14일): 5715원
4주(28일): 4643원
12주(84일): 4167원


In [37]:
# 시간권 이용금액 환산 함수
h4 = 4
h50 = 50
h100 = 100

def t_convert_amount(df, ticket_name, hours):
    price = int(math.ceil(price_table[price_table['이용권']==ticket_name]['가격'] / hours))
    return price

def t_convert_amount_ex(df, ticket_name, hours):
    price_extension = int(math.ceil(price_table[price_table['이용권']==ticket_name]['연장가'] / hours))
    return price_extension

In [39]:
print(f'4시간권: {t_convert_amount(price_table, "4시간", h4)}원')
print(f'50시간권: {t_convert_amount(price_table, "50시간(8주)", h50)}원')
print(f'100시간권: {t_convert_amount(price_table, "100시간(16주)", h100)}원')

4시간권: 1250원
50시간권: 1400원
100시간권: 1200원


- 이용권 가격대비 최소 이용시간 계산
    - 기준 이용권: 4주 기간권 기준
    - 이용권 가격: 130,000원 (4주 기간권의 신규 결제시 이용금액)
    - 시간당 요금: 1,200원 (100시간권의 1시간 이용금액, 시간권의 1시간 환산 이용금액 중 제일 낮은 금액을 사용)
- 최소 총 이용시간
    ```
    최소 총 이용시간 = 이용권 가격 / 시간당 요금
    ```

- 일일 최소 이용시간
    ```
    일일 최소 이용시간 = 이용권 가격 / (이용가능일수 X 시간당 요금)
    ```


In [104]:
# 2주 기간권 이용 시
# 최소 총 이용시간
print(price_table[price_table['이용권']=='2주(14일)']['가격'].values[0] / t_convert_amount(price_table, "100시간(16주)", h100))

# 일일 최소 이용시간
print(price_table[price_table['이용권']=='2주(14일)']['가격'].values[0] / (w2 * t_convert_amount(price_table, "100시간(16주)", h100)))

# 2주 기간권 기준, 총 66시간 40분이상, 하루에 4시간 45분이상 이용하면 된다.

66.66666666666667
4.761904761904762


In [105]:
# 4주 기간권 이용 시
# 최소 총 이용시간
print(price_table[price_table['이용권']=='4주(28일)']['가격'].values[0] / t_convert_amount(price_table, "100시간(16주)", h100))

# 일일 최소 이용시간
print(price_table[price_table['이용권']=='4주(28일)']['가격'].values[0] / (w4 * t_convert_amount(price_table, "100시간(16주)", h100)))

# 4주 기간권 기준, 총 108시간 20분이상, 하루에 3시간 52분이상 이용하면 된다.

108.33333333333333
3.869047619047619


In [106]:
# 12주 기간권 이용 시
# 최소 총 이용시간
print(price_table[price_table['이용권']=='12주(84일)']['가격'].values[0] / t_convert_amount(price_table, "100시간(16주)", h100))

# 일일 최소 이용시간
print(price_table[price_table['이용권']=='12주(84일)']['가격'].values[0] / (w12 * t_convert_amount(price_table, "100시간(16주)", h100)))

# 12주 기간권 기준, 총 291시간 40분이상, 하루에 3시간 28분이상 이용하면 된다.

291.6666666666667
3.4722222222222223


In [132]:
# 기준이 되는 변수 정의
# 최소 총 이용시간 기준
min_total_usage = (price_table[price_table['이용권']=='4주(28일)']['가격'].values[0] / t_convert_amount(price_table, "100시간(16주)", h100)).round(2)

# 일일 최소 이용시간 기준
min_daily_usage = (price_table[price_table['이용권']=='4주(28일)']['가격'].values[0] / (w4 * t_convert_amount(price_table, "100시간(16주)", h100))).round(2)

## 검증

In [167]:
# 검증 함수 정의
def verify(df):
    # 최소 총 이용시간 검증
    for index, row in df.iterrows():
        if row['총 이용시간_hours'] >= min_total_usage:
            print(f"{row['이용권']} total: good")
        else:
            print(f"{row['이용권']} total: cheer up")

    # 일일 최소 이용시간 검증
    for index, row in df.iterrows():
        if row['평균 이용시간_hours'] >= min_daily_usage:
            print(f"{row['이용권']} daily: good")
        else:
            print(f"{row['이용권']} daily: cheer up")

In [168]:
verify(summary)

시간권_4시간_1 total: cheer up
시간권_4시간_2 total: cheer up
시간권_50시간_1 total: cheer up
기간권_4주_1 total: good
기간권_4주_2 total: good
기간권_4주_3 total: cheer up
시간권_4시간_1 daily: good
시간권_4시간_2 daily: good
시간권_50시간_1 daily: good
기간권_4주_1 daily: good
기간권_4주_2 daily: good
기간권_4주_3 daily: good


# 결론

- 4주 기간권을 사용한다면 4주간 총 최소 약 108시간 20분, 하루에 최소 약 3시간 52분 이용하면 이용권을 잘 활용한다고 볼 수 있다.

- 4주 기간권을 구매한 1회차, 2회차의 기록과 비교했을 때 총 이용시간, 하루 평균 이용시간이 최소 이용시간보다 많기 때문에 나의 이용패턴에 맞는 이용권을 선택했다고 볼 수 있다.

# 회고

- 실생활과 관련있는 주제를 설정하여 직접 나의 데이터를 수집하고 분석하는 점이 흥미로웠다.
- 데이터 수집 과정에서 어떤 컬럼을 어떤 형식으로 수집하는 것이 보기에 편한 지, 데이터를 다루기에 좋을 지 등 고민하는 과정을 통해 데이터 수집 단계에서 설계의 중요성을 알게 되었다.
- 총 이용시간을 연산하는 부분이 쉽지 않았다.
- 데이터 타입을 필요에 따라 계속 변경하는 작업이 번거로웠다.
- 외출시간에 대한 데이터가 확보된다면 순수한 이용시간을 계산해 볼 수 있을 것이다.


# 분석 이후 기간 이용권 검증

- 5월 14일까지의 데이터를 추가 수집해 이용권을 기준 시간보다 많이 사용하고 있는지 검증한다.

In [169]:
# 분석 이후 추가된 데이터 불러오기
recent_data = pd.read_csv('/content/drive/MyDrive/personal_da_project/스터디카페 이용기록 - 이용기록_update.csv')
recent_payment = pd.read_csv('/content/drive/MyDrive/personal_da_project/스터디카페 이용기록 - 결제정보_update.csv')

In [170]:
recent_data.tail(), recent_payment.tail()

(          이용권          날짜      입실시간      퇴실시간 외출여부
 107  기간권_4주_5  2025-05-10  10:44:40  17:51:07    o
 108  기간권_4주_5  2025-05-11  12:15:23  18:19:37    x
 109  기간권_4주_5  2025-05-12  10:03:29  17:30:39    o
 110  기간권_4주_5  2025-05-13   9:34:18  18:49:37    o
 111  기간권_4주_5  2025-05-14   9:22:16  17:59:14    o,
       결제이용권        결제일자                 시작일시                 종료일시     결제금액  \
 3  기간권_4주_1  2025-01-13  2025-01-13 11:21:00  2025-02-10 11:21:00  130,000   
 4  기간권_4주_2  2025-02-09  2025-02-10 11:21:00  2025-03-10 11:21:00  120,000   
 5  기간권_4주_3  2025-03-05  2025-03-10 11:21:00  2025-04-07 11:21:00   99,000   
 6  기간권_4주_4  2025-04-07  2025-04-07 11:21:00  2025-05-05 11:21:00   120000   
 7  기간권_4주_5  2025-05-05  2025-05-05 11:21:00  2025-06-02 11:21:00   104000   
 
      이용권금액           비고  
 3  130,000          NaN  
 4  120,000         연장결제  
 5  109,000  할인이벤트, 쿠폰적용  
 6   120000         연장결제  
 7   109000  할인이벤트, 쿠폰적용  )

In [171]:
recent_data.info(), recent_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이용권     112 non-null    object
 1   날짜      112 non-null    object
 2   입실시간    112 non-null    object
 3   퇴실시간    112 non-null    object
 4   외출여부    112 non-null    object
dtypes: object(5)
memory usage: 4.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이용권     112 non-null    object
 1   날짜      112 non-null    object
 2   입실시간    112 non-null    object
 3   퇴실시간    112 non-null    object
 4   외출여부    112 non-null    object
dtypes: object(5)
memory usage: 4.5+ KB


(None, None)

In [172]:
# 데이터 타입 변경
recent_data['날짜'] = pd.to_datetime(recent_data['날짜'], format='%Y-%m-%d')
recent_data['입실시간'] = pd.to_datetime(recent_data['입실시간'], format='%H:%M:%S', errors='coerce')
recent_data['퇴실시간'] = pd.to_datetime(recent_data['퇴실시간'], format='%H:%M:%S', errors='coerce')

# 파생변수 생성 - 이용시간
recent_data['이용시간'] = recent_data['퇴실시간'] - recent_data['입실시간']

In [174]:
recent_data.head()

,이용권,날짜,입실시간,퇴실시간,외출여부,이용시간
0,시간권_4시간_1,2024-12-24,1900-01-01 13:21:28,1900-01-01 17:21:28,x,0 days 04:00:00
1,시간권_4시간_2,2024-12-27,1900-01-01 13:59:13,1900-01-01 17:59:13,x,0 days 04:00:00
2,시간권_50시간_1,2024-12-30,1900-01-01 13:45:57,1900-01-01 19:33:57,x,0 days 05:48:00
3,시간권_50시간_1,2025-01-02,1900-01-01 11:22:07,1900-01-01 19:52:33,x,0 days 08:30:26
4,시간권_50시간_1,2025-01-03,1900-01-01 11:03:36,1900-01-01 18:09:16,x,0 days 07:05:40


In [175]:
# 새로운 요약 데이터프레임 생성
summary_update = create_summary_dataframe(recent_data, recent_payment, recent_data['이용권'].unique())
summary_update

,이용권,총 이용시간,평균 이용시간,평균 입실시간,평균 퇴실시간,이용일수,"출석률(%, 기간권)",외출횟수,결제금액,총 이용시간_hours,평균 이용시간_hours
0,시간권_4시간_1,04:00:00,04:00:00,13:21:28,17:21:28,1,3.57,0,"5,000",4.000000,4.000000
1,시간권_4시간_2,04:00:00,04:00:00,13:59:13,17:59:13,1,3.57,0,"5,000",4.000000,4.000000
2,시간권_50시간_1,50:04:26,05:33:49,12:15:18,17:49:08,9,32.14,0,"70,000",50.073889,5.563611
3,기간권_4주_1,150:24:29,06:32:22,12:55:16,19:27:38,23,82.14,8,"130,000",150.408056,6.539444
4,기간권_4주_2,137:56:49,06:16:13,11:36:17,17:52:30,22,78.57,4,"120,000",137.946944,6.270278
5,기간권_4주_3,167:58:51,07:59:56,11:05:56,19:05:53,21,75.00,15,"99,000",167.980833,7.998889
6,기간권_4주_4,173:08:50,06:55:33,11:08:22,18:03:55,25,89.29,21,120000,173.147222,6.925833
7,기간권_4주_5,71:54:13,07:11:25,11:04:16,18:15:42,10,35.71,7,104000,71.903611,7.190278


In [176]:
# 새로운 데이터 검증하기
verify(summary_update)

시간권_4시간_1 total: cheer up
시간권_4시간_2 total: cheer up
시간권_50시간_1 total: cheer up
기간권_4주_1 total: good
기간권_4주_2 total: good
기간권_4주_3 total: good
기간권_4주_4 total: good
기간권_4주_5 total: cheer up
시간권_4시간_1 daily: good
시간권_4시간_2 daily: good
시간권_50시간_1 daily: good
기간권_4주_1 daily: good
기간권_4주_2 daily: good
기간권_4주_3 daily: good
기간권_4주_4 daily: good
기간권_4주_5 daily: good


- 기간권_4주_5의 경우 아직 사용 중이므로 총 이용시간이 기준에 못 미치는 것으로 나오지만 일일 평균 이용시간이 기준 시간보다 많으므로 이용권을 잘 이용하고 있음을 알 수 있다.